In [1]:
#確定點擊位置
import pyautogui
import time
print("請將滑鼠移動到目標按鈕，5 秒後座標會顯示在終端機...")
time.sleep(5)
print(pyautogui.position())  # 取得滑鼠當前位置

請將滑鼠移動到目標按鈕，5 秒後座標會顯示在終端機...
Point(x=733, y=600)


In [ ]:
import pyautogui
import time

# 設定 CSV 下載按鈕與日期箭頭的螢幕座標
# 請自行調整為您的實際座標
csv_download_button = (1517, 504)  # 替換成 CSV 下載按鈕的位置 (x, y)
next_date_arrow = (1072, 500)      # 替換成日期箭頭的位置 (x, y)

# 等待使用者準備（例如手動打開網站）
print("請打開目標網站並準備好，程式將在 5 秒後開始...")
time.sleep(5)

# 自動點擊流程
try:
    while True:
        # 點擊 CSV 下載按鈕
        pyautogui.click(csv_download_button[0], csv_download_button[1])
        print("已點擊 CSV 下載按鈕")
        time.sleep(0.8)  # 等待下載完成（根據需求調整時間）

        # 點擊日期右邊的箭頭
        pyautogui.click(next_date_arrow[0], next_date_arrow[1])
        print("已點擊日期箭頭")
        time.sleep(2)  # 等待頁面更新
except KeyboardInterrupt:
    print("操作已手動停止")


In [ ]:
#刪除指定欄位
import os
import glob
import pandas as pd

# 資料夾路徑
folder_path = "花蓮農改/"  # 修改為您的資料夾路徑

# 獲取資料夾中所有 CSV 文件的路徑
file_paths = sorted(glob.glob(os.path.join(folder_path, "*.csv")))

# 欲刪除的欄位名稱
columns_to_remove = ["TxSoil30cm"]

# 遍歷所有檔案
for file_path in file_paths:
    try:
        # 讀取文件，第一行作為標題
        data = pd.read_csv(file_path, header=0)

        # 刪除指定欄位
        data = data.drop(columns=[col for col in columns_to_remove if col in data.columns], errors='ignore')

        # 保存處理後的文件（覆蓋原文件）
        data.to_csv(file_path, index=False)
        print(f"Processed file: {file_path}")
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")


In [ ]:
#依照指定欄位合併檔案
import os
import glob
import pandas as pd

# 資料夾路徑，包含所有 CSV 檔案
folder_path = "C1Z120/"  # 假設所有文件都在此資料夾中

# 獲取資料夾中所有 CSV 文件的路徑
file_paths = sorted(glob.glob(os.path.join(folder_path, "*.csv")))

# 自定義標題（根據檢查的資料定義標題）
# column_names = [
#     "Hour", "Station Pressure (hPa)", "Sea Level Pressure (hPa)", "Temperature (°C)",
#     "Dew Point (°C)", "Relative Humidity (%)", "Wind Speed (m/s)", "Wind Direction (°)",
#     "Max Wind Speed (m/s)", "Max Wind Direction (°)", "Precipitation (mm)",
#      "Sunshine Duration (h)", "Solar Radiation (MJ/m²)",
#     "Ground Temp (0cm)", "Ground Temp (5cm)", "Ground Temp (10cm)",
#     "Ground Temp (20cm)", "Ground Temp (50cm)",
#     "Ground Temp (100cm)"
# ]
column_names = ["Hour","Precp"]


# 初始化存放合併數據的 DataFrame
combined_data = pd.DataFrame(columns=["datetime"] + column_names[1:])

# Function to adjust hour to datetime
def adjust_time(hour, base_date):
    try:
        if pd.isna(hour) or hour == '':
            return None
        if int(hour) == 24:
            return (base_date + pd.Timedelta(days=1)).strftime('%Y-%m-%d 00:00:00')
        else:
            return f"{base_date.strftime('%Y-%m-%d')} {int(hour):02}:00:00"
    except ValueError:
        return None

# 遍歷資料夾中的每個文件並處理 
for file_path in file_paths:
    file_name = os.path.basename(file_path)
    try:
        # 提取檔名中的日期
        file_date = "-".join(file_name.split('-')[1:]).replace('.csv', '').strip()
        base_date = pd.to_datetime(file_date, format='%Y-%m-%d')
    except (ValueError, IndexError) as e:
        print(f"Error parsing file date: {file_path}, Error message: {e}")
        continue  # 跳過無法解析的檔案

    # 讀取文件，跳過前兩行，並使用標準標題
    data = pd.read_csv(file_path, skiprows=1, names=column_names, header=None)

    # 生成 datetime 列
    data['datetime'] = data['Hour'].apply(lambda x: adjust_time(x, base_date))

    # 刪除原始 Hour 列
    data = data.drop(columns=['Hour'])

    # 確保 datetime 為第一列
    data = data[['datetime'] + [col for col in data.columns if col != 'datetime']]

    # 合併數據
    combined_data = pd.concat([combined_data, data], ignore_index=True)

# 保存合併結果
output_path = "combined_data_花蓮農改.csv"
combined_data.to_csv(output_path, index=False)

print(f"Data combined successfully. File saved at: {output_path}")


In [ ]:
#缺漏資料空白，取用所有欄位
# 修正程式碼以處理 24:00 時的情況，將其轉換為隔一天的 12:00 AM
import os
import glob
import pandas as pd
from datetime import datetime, timedelta

# 資料夾路徑
folder_path = "東華/"  # 修改為您的資料夾路徑

# 獲取資料夾中所有 CSV 文件的路徑
file_paths = sorted(glob.glob(os.path.join(folder_path, "*.csv")))

# 初始化空的 DataFrame
combined_data = pd.DataFrame()

# Function to create a datetime column in the specified format
def create_datetime(row, file_date):
    try:
        obs_hour = int(row['ObsTime'])
        if obs_hour == 24:
            # 24:00 表示隔天 12:00 AM
            datetime_obj = datetime.strptime(file_date, "%Y-%m-%d") + timedelta(days=1)
            return datetime_obj.strftime("%Y/%m/%d  12:00:00 AM")
        else:
            time_str = f"{file_date} {obs_hour:02}:00:00"
            datetime_obj = datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S")
            return datetime_obj.strftime("%Y/%m/%d  %I:%M:%S %p")
    except:
        return None

# 遍歷所有檔案
for file_path in file_paths:
    file_name = os.path.basename(file_path)
    # 從檔案名提取日期
    file_date = "-".join(file_name.split('-')[1:]).replace('.csv', '').strip()

    # 讀取文件
    try:
        data = pd.read_csv(file_path)
        # 使用第二行的英文欄位名稱作為標題
        data.columns = data.iloc[0]
        data = data.iloc[1:].reset_index(drop=True)

        # 生成 datetime 欄位
        data["datetime"] = data.apply(lambda row: create_datetime(row, file_date), axis=1)

        # 刪除原始 ObsTime 欄位
        data = data.drop(columns=["ObsTime"])

        # 確保 datetime 為第一列
        data = data[["datetime"] + [col for col in data.columns if col != "datetime"]]

        # 合併數據
        combined_data = pd.concat([combined_data, data], ignore_index=True)
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        continue

# 保存合併結果，確保 datetime 格式為字符串
output_path = os.path.join(folder_path, "combined_data_with_adjusted_datetime.csv")
combined_data.to_csv("combined_data_with_adjusted_datetime.csv", index=False)

print(f"合併完成，結果已保存至 {output_path}")
